# Chapter 09: Transforming Unstructured Data into Structured Formats

In today's data-driven world, companies are inundated with vast amounts of unstructured data, including documents, slides, emails, and more. Transforming this unstructured data into structured formats is crucial for unlocking its full potential. Leveraging LLMs enables businesses to efficiently convert unstructured information into datasets that can be stored in databases, CSV files, and other structured formats. This transformation not only facilitates data management but also allows for the training of machine learning models on the structured datasets. Ultimately, structured data can be seamlessly integrated into various workflows, functions, and processes, enabling more informed decision-making across the organization.

In [ ]:
from assets.tools.earthquake import count_earthquakes, query_earthquakes, USGeopoliticalSurveyEarthquakeAPI
from assets.tools.forecasting import forecast_earthquakes, get_regions
from pydantic import BaseModel, Field, EmailStr
from language_models.models.llm import OpenAILanguageModel
from language_models.agent import (
    Agent,
    Workflow,
    WorkflowLLMStep,
    WorkflowFunctionStep,
    OutputType,
    PromptingStrategy,
)
from language_models.tools import Tool, current_date
from language_models.proxy_client import ProxyClient
from language_models.settings import settings

In [ ]:
proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)